<a href="https://colab.research.google.com/github/AtharvaaPatil/Automatic-Question-Gen/blob/main/AQG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet transformers==2.9.0
!pip install --quiet nltk==3.4.5

     |████████████████████████████████| 635 kB 4.1 MB/s 
     |████████████████████████████████| 1.3 MB 51.8 MB/s 
     |████████████████████████████████| 5.6 MB 26.5 MB/s 
     |████████████████████████████████| 880 kB 26.4 MB/s 
     |████████████████████████████████| 1.5 MB 2.7 MB/s 


In [4]:
!sudo apt-get install -qq w3m # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libgpm2:amd64.
(Reading database ... 155685 files and directories currently installed.)
Preparing to unpack .../libgpm2_1.20.7-5_amd64.deb ...
Unpacking libgpm2:amd64 (1.20.7-5) ...
Selecting previously unselected package w3m.
Preparing to unpack .../w3m_0.5.3-36build1_amd64.deb ...
Unpacking w3m (0.5.3-36build1) ...
Setting up libgpm2:amd64 (1.20.7-5) ...
Setting up w3m (0.5.3-36build1) ...
Processing triggers for mime-support (3.60ubuntu1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.5) ...
Proces

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**Generating Distractors (wrong choices for MCQ options)**

In [7]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

sentence1 = "Srivatsan loves to watch cricket during his free time"
sentence2 = "Srivatsan is annoyed by a cricket in his room"

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [8]:
# Example of a word with two different senses
original_word = "cricket"

syns = wn.synsets(original_word,'n')

for syn in syns:
  print (syn, ": ",syn.definition(),"\n")

Synset('cricket.n.01') :  leaping insect; male makes chirping noises by rubbing the forewings together 

Synset('cricket.n.02') :  a game played with a ball and bat by two teams of 11 players; teams take turns trying to score runs 



In [10]:
!git status

fatal: not a git repository (or any of the parent directories): .git
